In [ ]:
#installing pytorch
!uv add torch torchvision

Resolved 29 packages in 207ms                                        
⠙ Preparing packages... (0/2)                                                   
⠙ Preparing packages... (0/2)--------------     0 B/5.13 MiB            
⠙ Preparing packages... (0/2)--------------     0 B/5.13 MiB            
torchvision          ------------------------------     0 B/1.77 MiB
⠙ Preparing packages... (0/2)--------------     0 B/5.13 MiB            
torchvision          ------------------------------     0 B/1.77 MiB
⠙ Preparing packages... (0/2)-------------- 16.00 KiB/5.13 MiB          
torchvision          ------------------------------     0 B/1.77 MiB
⠙ Preparing packages... (0/2)-------------- 32.00 KiB/5.13 MiB          
torchvision          ------------------------------     0 B/1.77 MiB
⠙ Preparing packages... (0/2)-------------- 48.00 KiB/5.13 MiB          
torchvision          ------------------------------     0 B/1.77 MiB
⠙ Preparing packages... (0/2)-------------- 62.40 KiB/5.13 MiB    

In [4]:
!uv add opencv-python

Resolved 30 packages in 464ms                                        
⠙ Preparing packages... (0/2)                                                   
⠙ Preparing packages... (0/2)--------------     0 B/36.12 MiB           
⠙ Preparing packages... (0/2)--------------     0 B/36.12 MiB           
numpy                ------------------------------     0 B/5.10 MiB
⠙ Preparing packages... (0/2)--------------     0 B/36.12 MiB           
numpy                ------------------------------ 16.00 KiB/5.10 MiB
⠙ Preparing packages... (0/2)--------------     0 B/36.12 MiB           
numpy                ------------------------------ 32.00 KiB/5.10 MiB
⠙ Preparing packages... (0/2)--------------     0 B/36.12 MiB           
numpy                ------------------------------ 48.00 KiB/5.10 MiB
⠙ Preparing packages... (0/2)--------------     0 B/36.12 MiB           
numpy                ------------------------------ 64.00 KiB/5.10 MiB
⠙ Preparing packages... (0/2)--------------     0 B/36.12 

In [2]:
#importing std dependencies
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt


In [3]:
#importing pytorch dependencies
import torch
from torchmetrics.classification import Accuracy, Precision, Recall

In [42]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [4]:
#creating folders
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')
'''os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)'''

'os.makedirs(POS_PATH)\nos.makedirs(NEG_PATH)\nos.makedirs(ANC_PATH)'

moving lfw images to negative folder

In [8]:
for directory in os.listdir('lfw_funneled'):
    if os.path.isdir(os.path.join('lfw_funneled',directory,file)):
            for file in os.listdir('lfw_funneled'+'/'+directory):
        
                
                EX_PATH = os.path.join('lfw_funneled',directory,file)
                NEW_PATH = os.path.join(NEG_PATH,file)
                os.replace(EX_PATH,NEW_PATH)

collecting anchor and positive images

In [8]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    # cutting frame to 250 by 250px
    frame = frame[0:250, 0:250 ,:]

    cv2.imshow("frame", frame)
    #collecting anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        pass

    #collecting positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
      pass

    if cv2.waitKey(1) & 0XFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Read image (OpenCV loads as BGR)
img = cv2.imread('data/anchor/447a4b68-acb7-11f0-b160-221fedd10dee.jpg')

# Convert BGR → RGB for correct display in matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Show in notebook
plt.imshow(img_rgb)
plt.show()


Getting image directories


In [5]:
from torch.utils.data import Dataset, DataLoader
import glob


# List all matching files and take the first 300
anchor_files = glob.glob(ANC_PATH + '/*.jpg')[:300]
positive_files = glob.glob(POS_PATH + '/*.jpg')[:300]
negative_files = glob.glob(NEG_PATH + '/*.jpg')[:300]

#creating custom dataset
class FilePathDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        return self.file_list[idx]


anchor = FilePathDataset(anchor_files)
postive = FilePathDataset(positive_files)
negative = FilePathDataset(negative_files)

#creating dataloader
dir_test = DataLoader(anchor, batch_size=1)

dir_test_iter = iter(dir_test)



In [7]:
postive.__len__()

300

In [8]:
print(next(dir_test_iter))

['data/anchor/447a4b68-acb7-11f0-b160-221fedd10dee.jpg']


Preprocessing images

In [6]:
from torchvision.io import decode_image,read_file
import torchvision

from PIL import Image
import torchvision.transforms as transforms

'''def preprocess(img_path):
    image = Image.open(img_path).convert('RGB')
    image = transforms.Resize((100, 100))(image)
    image = transforms.ToTensor()(image)
    return image'''

def preprocess(img_path):
    img_bytes = read_file(img_path)
    image = decode_image(img_bytes)
    image = torchvision.transforms.Resize((100,100))(image)
    image = image / 255.0
    return image

#example 
'''img = preprocess("data/anchor/61434828-a2b0-11f0-b077-221fedd10dee.jpg")
plt.imshow(img.permute(1,2,0))
plt.show()'''

'img = preprocess("data/anchor/61434828-a2b0-11f0-b077-221fedd10dee.jpg")\nplt.imshow(img.permute(1,2,0))\nplt.show()'

In [8]:
def preprocess_twin(input_img,validation_img,label):
    input_img = preprocess(input_img)
    validation_img = preprocess(validation_img)
    return (input_img,validation_img,label)

Creating labelled dataset


In [9]:
from torch.utils.data import Dataset, DataLoader
import glob

# Example paths
anchor_files = glob.glob(ANC_PATH + '/*.jpg')[:57]
positive_files = glob.glob(POS_PATH + '/*.jpg')[:57]
negative_files = glob.glob(NEG_PATH + '/*.jpg')[:57]


# Dataset that returns (anchor_path, other_path, label)
class PairedFilePathDataset(Dataset):
    def __init__(self, anchor_files, other_files, label,transform = None):
        assert len(anchor_files) == len(other_files), "Anchor and other files must match in length"
        self.anchor_files = anchor_files
        self.other_files = other_files
        self.label = label
        self.transform = transform
        self.cache = {}  # in-memory caching

    def __len__(self):
        return len(self.anchor_files)

    def __getitem__(self, idx):
        if idx in self.cache:
            return self.cache[idx]
        anchor_path = self.anchor_files[idx]
        other_path = self.other_files[idx]
        label_tensor = torch.tensor(self.label, dtype=torch.float32)
         # Apply preprocessing function here
        if self.transform:
            anchor_path, other_path,label_tensor = self.transform(anchor_path, other_path,label_tensor)
        sample = (anchor_path, other_path, label_tensor)
        self.cache[idx] = sample  # cache it in memory
        return sample


# Create positive and negative datasets
positive_dataset = PairedFilePathDataset(anchor_files, positive_files, label=1,transform=preprocess_twin)
negative_dataset = PairedFilePathDataset(anchor_files, negative_files, label=0,transform=preprocess_twin)

# Combine datasets manually (no ConcatDataset nesting)
full_dataset = positive_dataset + negative_dataset  # works because __getitem__ returns tuples


# Custom collate function to return tuple directly
def single_collate_fn(batch):
    # batch is a list of 1 item if batch_size=1
    anchor, other, label = batch[0]
    return anchor, other, label


# DataLoader
paired_dataloader = DataLoader(
    full_dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=single_collate_fn
)

# Example usage
example = next(iter(paired_dataloader))
example


(tensor([[[0.6706, 0.6706, 0.6706,  ..., 0.4118, 0.4078, 0.4078],
          [0.6745, 0.6745, 0.6745,  ..., 0.4078, 0.4078, 0.4157],
          [0.6784, 0.6784, 0.6745,  ..., 0.4118, 0.4157, 0.4235],
          ...,
          [0.2431, 0.2000, 0.2000,  ..., 0.3333, 0.3529, 0.3765],
          [0.1725, 0.2000, 0.2471,  ..., 0.4667, 0.3843, 0.3608],
          [0.1922, 0.2745, 0.2863,  ..., 0.5922, 0.5765, 0.5020]],
 
         [[0.5647, 0.5647, 0.5647,  ..., 0.3529, 0.3569, 0.3490],
          [0.5686, 0.5686, 0.5686,  ..., 0.3529, 0.3569, 0.3569],
          [0.5725, 0.5725, 0.5686,  ..., 0.3529, 0.3608, 0.3569],
          ...,
          [0.2392, 0.1922, 0.2000,  ..., 0.4118, 0.4118, 0.4275],
          [0.1686, 0.1961, 0.2431,  ..., 0.5686, 0.4863, 0.4588],
          [0.1922, 0.2745, 0.2824,  ..., 0.6784, 0.6745, 0.6039]],
 
         [[0.4980, 0.4980, 0.4980,  ..., 0.2980, 0.2941, 0.2902],
          [0.5020, 0.5020, 0.5020,  ..., 0.3020, 0.2980, 0.2980],
          [0.5059, 0.5059, 0.5020,  ...,

Building train and test partitions

In [13]:
len(example)

3

In [ ]:
plt.imshow(example[0].permute(1,2,0))

Building a data pipeline

In [10]:
from torch.utils.data import random_split
total_len = len(full_dataset)
train_len = int(0.7 * total_len)
test_len = total_len - train_len
train_dataset, test_dataset = random_split(full_dataset, [train_len, test_len])

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True      # shuffles the dataset every epoch
)

test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False
)


Building an embedding layer 

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 10)
        self.conv2 = nn.Conv2d(64, 128, 7)
        self.conv3 = nn.Conv2d(128, 128, 4)
        self.conv4 = nn.Conv2d(128, 256, 4)

        self.fc1 = nn.Linear(256 * 5 * 5, 4096)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.fc1(x))  
        return x

model = Embedding()
model

Embedding(
  (conv1): Conv2d(3, 64, kernel_size=(10, 10), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=6400, out_features=4096, bias=True)
)

Creating siamese L1 distance layer

In [12]:
class L1Dist(nn.Module):
    def __init__(self)->None:
        super().__init__()
    
    def forward(self,input_embedding,validation_embedding):
        return torch.abs(input_embedding - validation_embedding)


Creating siamese model

In [13]:
class SiameseNeuralNetwork(nn.Module):
    def __init__(self):
        super(SiameseNeuralNetwork, self).__init__()
        self.embedding = Embedding()
        self.L1Dist = L1Dist()
        self.fc1 = nn.Linear(4096,1)

    def forward(self,input_img,validation_img):
        input_embedding = self.embedding(input_img)
        validation_embedding = self.embedding(validation_img)

        l1_distance = self.L1Dist(input_embedding,validation_embedding)
        output = torch.sigmoid(self.fc1(l1_distance))
        return output

model = SiameseNeuralNetwork()

Creating Train step model

In [51]:
batch_1 = next(iter(test_loader))

In [22]:
batch_1[2]

tensor([1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.])

setting up checkpoints

In [23]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

setting up loss and optimizer 

In [24]:
lossfn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)

Training function

In [25]:
def train_one_epoch(batch):
    model.train()
    anchor_img,validation_img,label = batch
    label = label.unsqueeze(1)
    optimizer.zero_grad()
    output = model(anchor_img,validation_img)
    loss = lossfn(output,label)
    print(loss)
    loss.backward()
    optimizer.step()
    return loss

Training loop

In [26]:
def train(data,epochs):
    for epoch in range(1,epochs+1):
        print('\n Epoch {}/{}'.format(epoch, epochs))
        for batch in data:
            loss = train_one_epoch(batch)
            print(f"Epoch {epoch+1} Loss: {loss}")

        if epoch % 10 == 0:
            torch.save(model.state_dict(), f"{checkpoint_prefix}/siamese_epoch_{epoch}.pth")
            print(f"Model saved at epoch {epoch}")

Training Model

In [27]:
epochs = 50
train(train_loader,epochs)


 Epoch 1/50
tensor(0.6941, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 2 Loss: 0.6941414475440979
tensor(0.6898, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 2 Loss: 0.6897565126419067
tensor(0.6810, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 2 Loss: 0.6810295581817627
tensor(0.6646, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 2 Loss: 0.6645702123641968
tensor(0.6809, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 2 Loss: 0.6809360980987549

 Epoch 2/50
tensor(0.6460, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 3 Loss: 0.646033763885498
tensor(0.5600, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 3 Loss: 0.5599795579910278
tensor(0.5813, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 3 Loss: 0.5812582969665527
tensor(0.4297, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 3 Loss: 0.4297126531600952
tensor(0.5201, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 3 Loss: 0.5201035141944885

 Epoch 3/50
tensor(0.3565, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 4 Loss: 0.3565317988395691
tenso

Evaluating model and making visualizations

In [52]:
#importing metrics
from sklearn.metrics import precision_score,recall_score
test_input, test_val, test_label = batch_1
y_pred = model(test_input,test_val)
y_pred

tensor([[0.5023],
        [0.5023],
        [0.5024],
        [0.5023],
        [0.5023],
        [0.5024],
        [0.5024],
        [0.5023],
        [0.5024],
        [0.5023],
        [0.5023],
        [0.5023],
        [0.5024],
        [0.5022],
        [0.5025],
        [0.5023]], grad_fn=<SigmoidBackward0>)

In [35]:
y_pred = torch.tensor([1 if prediction > 0.5 else 0 for prediction in y_pred])
y_pred

tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1])

In [30]:
test_label

tensor([1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.])

In [ ]:
#calculating metrics
p = Precision(task="binary")
a = Accuracy(task="binary")
r = Recall(task="binary")
precision = p(y_pred,test_label)
accuracy = a(y_pred,test_label)
recall = r(y_pred,test_label)
print("precision:",precision.item())
print("accuracy:",accuracy.item())
print("recall:",recall.item())


precision: 1.0
accuracy: 1.0
recall: 1.0


Visualizing Results

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.imshow(test_input[2].permute(1,2,0))
plt.subplot(1,2,2)
plt.imshow(test_val[2].permute(1,2,0))
plt.show()


Saving model

In [50]:
torch.save(model, 'siamese_model.pth')

In [14]:
#reloading the model 
model = torch.load('siamese_model.pth',weights_only=False)
model.eval()


SiameseNeuralNetwork(
  (embedding): Embedding(
    (conv1): Conv2d(3, 64, kernel_size=(10, 10), stride=(1, 1))
    (conv2): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1))
    (conv3): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1))
    (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(1, 1))
    (fc1): Linear(in_features=6400, out_features=4096, bias=True)
  )
  (L1Dist): L1Dist()
  (fc1): Linear(in_features=4096, out_features=1, bias=True)
)

In [54]:
#predicting 
prediction = model(test_input,test_val)
prediction

tensor([[1.0000e+00],
        [4.7314e-04],
        [1.2407e-05],
        [9.9770e-01],
        [6.6275e-07],
        [2.9213e-04],
        [4.5840e-06],
        [9.8801e-01],
        [9.9987e-01],
        [5.6815e-05],
        [9.9977e-01],
        [9.9833e-01],
        [8.0892e-06],
        [9.9998e-01],
        [3.5733e-07],
        [1.0000e+00]], grad_fn=<SigmoidBackward0>)

Real time test

In [15]:
def verify(model,detection_threshold,verification_threshold):
    results = []
    for image in os.listdir(os.path.join('application_data','verification_images')):
        if(image.endswith('.jpg')==False):
            continue
        input_img = preprocess(os.path.join('application_data','input_image','input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data','verification_images',image))
        
        input_img = input_img.unsqueeze(0)
        validation_img = validation_img.unsqueeze(0)
        
        output = model(input_img,validation_img)
        results.append(output.item())

        #detection threshold: tells us is above which value we should consider it to be a positive match
        #verification threshold: tells us what percentage of verification images should match to consider it verified
        detection = np.sum(np.array(results) > detection_threshold)
        verification = detection / len(os.listdir(os.path.join('application_data','verification_images')))
        verified = verification > verification_threshold
    return results, verified

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    # cutting frame to 250 by 250px
    frame = frame[275:275+250, 550:550+250, :]

    cv2.imshow("frame", frame)
    #saving input image
    if cv2.waitKey(1) & 0XFF == ord('v'):
        input_img_path = os.path.join('application_data','input_image','input_image.jpg')
        cv2.imwrite(input_img_path, frame)
        results, verified = verify(model,0.5,0.5)
        print(f"verified: {verified}, results: {results}")

    if cv2.waitKey(1) & 0XFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()

verified: True, results: [0.8468645215034485, 0.008444461040198803, 0.824988067150116, 0.0011397842317819595, 0.008139001205563545, 0.8528907895088196, 0.007038509007543325, 0.9904754757881165, 0.0006823272560723126, 0.0022322151344269514, 0.008792937733232975, 0.7709122896194458, 0.8586593270301819, 0.8890367746353149, 0.49415865540504456, 0.8322484493255615, 0.9315831065177917, 0.0007346792845055461, 0.001885408302769065, 0.9753064513206482, 0.006021004170179367, 0.000743978947866708, 0.011461793445050716, 0.9525134563446045, 0.9607352614402771, 0.44182583689689636, 0.8229920268058777, 0.8953891396522522, 0.978950560092926, 0.034712791442871094, 0.0005765804089605808, 0.007391834631562233, 0.6802200078964233, 0.8940867781639099, 0.0011372192529961467, 0.005828074179589748, 0.9565826654434204, 0.8991504311561584, 0.8479660153388977, 0.9965894222259521, 0.9445160627365112, 0.9728336930274963, 0.8012357950210571, 0.9980168342590332, 0.0030442841816693544, 0.009657356888055801, 0.9066675

: 